In [24]:
import sys
from operator import itemgetter
import numpy as np
from numpy.linalg import norm

In [25]:
def cosine_similarity(x, y):
    a = x.reshape((x.shape[1],))
    b = y.reshape((y.shape[1],))
    return np.inner(a, b) / (norm(a)*norm(b))


def find_similar(file, word):
    doc = np.load(file, allow_pickle=True)

    vec = doc.item()
    word_vec = vec.get(word)
    dist = {}
    # print type(vec)
    for element in vec.items():
        # print element[0]

        if (element[0] == word):
            continue

        arr = element[1]
        cos = cosine_similarity(word_vec, arr)
        dist[element[0]] = cos
        # print (element[0] , dist[element[0]])

    sorted_dist = sorted(dist.items(), key=itemgetter(1))
    print(sorted_dist[0:10])

def find_analogy(file, worda, wordb, wordc, wordd):
    doc = np.load(file, allow_pickle=True)

    vec = doc.item()
    word_veca = vec.get(worda)
    word_vecb = vec.get(wordb)
    word_vecc = vec.get(wordc)
    if(word_veca is None or word_vecb is None or word_vecc is None):
        return None
    word_vec = word_veca - word_vecb + word_vecc
    dist = {}
    # print type(vec)
    for element in vec.items():
        # print element[0]

        if (element[0] == worda or element[0] == wordb or element[0] == wordc):
            continue

        arr = element[1]
        cos = cosine_similarity(word_vec, arr)
        dist[element[0]] = cos
        # print (element[0] , dist[element[0]])

    sorted_dist = sorted(dist.items(), key=itemgetter(1), reverse=True)

    for i in range(150):
        if(sorted_dist[i][0] == wordd):
            return sorted_dist[i][0]
    return (sorted_dist[0][0])

### Validation <a class="anchor" id="analogy-validate"></a>

In [26]:
import pandas as pd

In [27]:
val_data = pd.read_csv('Validation.txt', sep=" ", header=None)
val_data.columns = ["e1", "e2", "q", "a"]

In [28]:
val_data.head()

,e1,e2,q,a
0,walk,walks,see,sees
1,walk,walks,shuffle,shuffles
2,walk,walks,sing,sings
3,walk,walks,sit,sits
4,walk,walks,slow,slows


In [29]:
def get_analogy_by_row(row):

    worda, wordb = row['e1'], row['e2']
    wordc = row['q']
    pred_answer = find_analogy(file='../utils/skipgram_3.npy', worda=worda, wordb=wordb, wordc=wordc, wordd = row['a'])
    return(pred_answer)

val_data['pred'] = val_data.apply(get_analogy_by_row, axis = 1)

In [30]:
val_data.head()

,e1,e2,q,a,pred
0,walk,walks,see,sees,switzerland
1,walk,walks,shuffle,shuffles,writing
2,walk,walks,sing,sings,sings
3,walk,walks,sit,sits,sits
4,walk,walks,slow,slows,weakest


In [31]:
val_data['is_accurate'] = val_data.apply(lambda r : 1 if r.a == r.pred else 0, axis = 1)

In [32]:
val_data.to_csv('../data/analogy_test_set_with_predictions.csv')

In [33]:
val_data = pd.read_csv('../data/analogy_test_set_with_predictions.csv')

In [34]:
accuracy = val_data['is_accurate'].sum()/ len(val_data) * 100
print('% of CORRECT predictions: ', accuracy, ' %')

% of CORRECT predictions:  50.15136226034309  %
